<a href="https://colab.research.google.com/github/adity1234567/Deep-Dive-in-The-Field-of-Misinformation/blob/main/Deep_Learning_For_NLP_Zero_To_Transformers_%26_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/tanulsingh077/deep-learning-for-nlp-zero-to-transformers-bert

Contents

In this Notebook I will start with the very Basics of RNN's and Build all the way to latest deep learning architectures to solve NLP problems. It will cover the Following:

- Simple RNN's

- Word Embeddings : Definition and How to get them
- LSTM's
- GRU's
- BI-Directional RNN's
- Encoder-Decoder Models (Seq2Seq Models)
- Attention Models
- Transformers - Attention is all you need
- BERT

In [11]:
import plotly.graph_objs as go  # for advanced customization of figures
import plotly.express as px     # for easier plotting
import plotly.figure_factory as ff  # for specialized charts


In [12]:
pip install tensorflow --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 858.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.3 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0

In [13]:
pip install numpy pandas scikit-learn tensorflow keras matplotlib seaborn plotly


In [14]:
import numpy as np


import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, GRU, SimpleRNN, Dense, Activation, Dropout, Embedding, BatchNormalization, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing import sequence, text

from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff


Configuring TPU's

For this version of Notebook we will be using TPU's as we have to built a BERT Model **bold text**

In [15]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [ ]:
!unzip '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/jigsaw-toxic-comment-train.csv.zip' -d '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/'
!unzip '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/validation.csv.zip' -d '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/'
!unzip '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/test.csv.zip' -d '/content/drive/MyDrive/Health misinformation/zero to ransfer bert/'

Archive:  /content/drive/MyDrive/Health misinformation/zero to ransfer bert/jigsaw-toxic-comment-train.csv.zip
replace /content/drive/MyDrive/Health misinformation/zero to ransfer bert/jigsaw-toxic-comment-train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/drive/MyDrive/Health misinformation/zero to ransfer bert/validation.csv.zip
  inflating: /content/drive/MyDrive/Health misinformation/zero to ransfer bert/validation.csv  
Archive:  /content/drive/MyDrive/Health misinformation/zero to ransfer bert/test.csv.zip
  inflating: /content/drive/MyDrive/Health misinformation/zero to ransfer bert/test.csv  


In [29]:
train = pd.read_csv('/content/drive/MyDrive/Health misinformation/zero to ransfer bert/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('/content/drive/MyDrive/Health misinformation/zero to ransfer bert/validation.csv')
test = pd.read_csv('/content/drive/MyDrive/Health misinformation/zero to ransfer bert/test.csv')


In [31]:
train.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


In [32]:
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [33]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)

In [34]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223549 entries, 0 to 223548
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            223549 non-null  object
 1   comment_text  223549 non-null  object
 2   toxic         223549 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 5.1+ MB


In [35]:

train=train.loc[:12000,:]
train.shape

(12001, 3)

We will check the maximum number of words that can be present in a comment , this will help us in padding later

This code helps find the longest comment (by word count) in the dataset.

In [36]:
train['comment_text'].apply(lambda x:len(str(x).split())).max()

1403

Writing a function for getting auc score for validation

In [37]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''

    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc


In [38]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values,
                                                  stratify=train.toxic.values,
                                                  random_state=42,
                                                  test_size=0.2, shuffle=True)

# using keras tokenizer here

In [47]:
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [49]:
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=300,
                    input_shape=(max_len,)))  # Use input_shape instead of input_length
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ (None, 1500, 300)           │      13,049,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 100)                 │          40,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,089,301 (49.93 MB)

 Trainable params: 13,089,301 (49.93 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64 * strategy.num_replicas_in_sync)


Epoch 1/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 366s 2s/step - accuracy: 0.8747 - loss: 0.3512
Epoch 2/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 373s 2s/step - accuracy: 0.9661 - loss: 0.1058
Epoch 3/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 382s 2s/step - accuracy: 0.9959 - loss: 0.0146
Epoch 4/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 383s 2s/step - accuracy: 1.0000 - loss: 0.0017
Epoch 5/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 383s 2s/step - accuracy: 1.0000 - loss: 5.4481e-04


In [1]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip


--2025-03-23 06:30:15--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2025-03-23 06:30:15--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2025-03-23 06:30:15--  https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/

In [3]:
!unzip '/content/glove.840B.300d.zip' -d '/content/drive/MyDrive/Health misinformation/glove'

Archive:  /content/glove.840B.300d.zip
  inflating: /content/drive/MyDrive/Health misinformation/glove/glove.840B.300d.txt  


In [44]:
from tqdm import tqdm  # Import tqdm for progress bar
import numpy as np
embeddings_index = {}
with open('/content/drive/MyDrive/Health misinformation/glove/glove.840B.300d.txt', 'r', encoding='utf-8') as f:
    for line in tqdm(f):  # Use tqdm to show progress bar
        values = line.split(' ')
        word = values[0]
        coefs = np.asarray([float(val) for val in values[1:]])
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))


2196017it [04:06, 8911.29it/s] 

Found 2196016 word vectors.


In [46]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 43496/43496 [00:00<00:00, 152574.61it/s]


In [52]:
%%time
with strategy.scope():

    # A simple LSTM with GloVe embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                        300,
                        weights=[embedding_matrix],
                        input_length=max_len,
                        trainable=True))  # Now, embeddings will be updated during training

    model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)              │ ?                           │      13,049,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,049,100 (49.78 MB)

 Trainable params: 13,049,100 (49.78 MB)

 Non-trainable params: 0 (0.00 B)

CPU times: user 232 ms, sys: 241 ms, total: 473 ms
Wall time: 459 ms


In [ ]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64 * strategy.num_replicas_in_sync)


Epoch 1/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 1203s 8s/step - accuracy: 0.9206 - loss: 0.2416
Epoch 2/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 1193s 8s/step - accuracy: 0.9612 - loss: 0.1166
Epoch 3/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 1237s 8s/step - accuracy: 0.9696 - loss: 0.0909
Epoch 4/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 1229s 8s/step - accuracy: 0.9799 - loss: 0.0568
Epoch 5/5
 63/150 ━━━━━━━━━━━━━━━━━━━━ 11:43 8s/step - accuracy: 0.9925 - loss: 0.0269

In [61]:
from sklearn.metrics import roc_auc_score

scores = model.predict(xvalid_pad)
auc = roc_auc_score(yvalid, scores)  # Compute AUC score
print("AUC: %.2f%%" % (auc * 100))


76/76 ━━━━━━━━━━━━━━━━━━━━ 75s 968ms/step
AUC: 96.82%


In [63]:
# Initialize scores_model list
scores_model = []

# Append the GRU model's AUC score
scores_model.append({'Model': 'GRU', 'AUC_Score': roc_auc_score(yvalid, scores)})

# Print the results
print(scores_model)


[{'Model': 'GRU', 'AUC_Score': np.float64(0.9681518466133601)}]


In [64]:
scores_model

[{'Model': 'GRU', 'AUC_Score': np.float64(0.9681518466133601)}]

In [66]:
%%time
with strategy.scope():
    # A simple bidirectional LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                    300,
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=True))  # Allow fine-tuning

    model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])


model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)             │ ?                           │      13,049,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,049,100 (49.78 MB)

 Trainable params: 13,049,100 (49.78 MB)

 Non-trainable params: 0 (0.00 B)

CPU times: user 250 ms, sys: 184 ms, total: 434 ms
Wall time: 309 ms


In [65]:
%%time
with strategy.scope():
    # A simple bidirectional LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
    model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])


model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)             │ ?                           │      13,049,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,049,100 (49.78 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 13,049,100 (49.78 MB)

CPU times: user 316 ms, sys: 293 ms, total: 609 ms
Wall time: 1.44 s
